# 1. Load

# The Problem: 
### Predicting whether a person with a given set of characteristics is likely to have a heart attack.

In [10]:
import pandas as pd
!pwd
df_raw = pd.read_csv("final-test/data/heart.csv") 
print("Columns:", df_raw.columns.tolist())

/Users/rosasampaio/Documents/github/data_science_prediction_pacient
Columns: ['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS', 'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope', 'HeartDisease']


In [11]:
print(df_raw.head(1))
print("Types :\n",df_raw.dtypes)

   Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  MaxHR  \
0   40   M           ATA        140          289          0     Normal    172   

  ExerciseAngina  Oldpeak ST_Slope  HeartDisease  
0              N      0.0       Up             0  
Types :
 Age                 int64
Sex                   str
ChestPainType         str
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG            str
MaxHR               int64
ExerciseAngina        str
Oldpeak           float64
ST_Slope              str
HeartDisease        int64
dtype: object


2. Pré-processamento

In [12]:
print(df_raw.isna())
print("SIZE: ", len(df_raw))

       Age    Sex  ChestPainType  RestingBP  Cholesterol  FastingBS  \
0    False  False          False      False        False      False   
1    False  False          False      False        False      False   
2    False  False          False      False        False      False   
3    False  False          False      False        False      False   
4    False  False          False      False        False      False   
..     ...    ...            ...        ...          ...        ...   
913  False  False          False      False        False      False   
914  False  False          False      False        False      False   
915  False  False          False      False        False      False   
916  False  False          False      False        False      False   
917  False  False          False      False        False      False   

     RestingECG  MaxHR  ExerciseAngina  Oldpeak  ST_Slope  HeartDisease  
0         False  False           False    False     False         False  

In [13]:
# same the df.shape[0]
df_raw.isna().count()


Age               918
Sex               918
ChestPainType     918
RestingBP         918
Cholesterol       918
FastingBS         918
RestingECG        918
MaxHR             918
ExerciseAngina    918
Oldpeak           918
ST_Slope          918
HeartDisease      918
dtype: int64

In [14]:
df_raw.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

Resultado: não existens ausentes

# 3. Normalization

In [15]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
# Normalization
# Apply StandardScaler ONLY to numeric columns.
# And you should NEVER try to transform categorical columns to 0 and 1 before standardizing.
# The same logic applies to OneHotEncoder (categorical columns ONLY)

df_categorical = df_raw.select_dtypes(exclude=['number'])
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded = encoder.fit_transform(df_categorical)

encoded_df = pd.DataFrame(
    encoded,
    columns=encoder.get_feature_names_out(df_categorical.columns),
    index=df_raw.index
)

# Outliers: Statistical Rule vs. Quality Rule (Domain Violation)
Fundamental Difference (Very Important)

## Statistical Outlier (IQR)
- Extreme but plausible value
- Ex.: Cholesterol = 450
- Action: cap / flag / feature engineering

## Domain Violation (Data Quality)
- Impossible value in the real world
- Ex.: Age = 200
- Action: quality rule, not statistical

*Age = 200 is not an outlier

*Age = 200 is invalid data

# Treat outliers only in the original numeric columns.

In [16]:
df_numeric = df_raw.select_dtypes(include=['number'])
df_numeric.dtypes

Age               int64
RestingBP         int64
Cholesterol       int64
FastingBS         int64
MaxHR             int64
Oldpeak         float64
HeartDisease      int64
dtype: object

In [17]:
print(df_numeric["RestingBP"].value_counts()) 

RestingBP
120    132
130    118
140    107
110     58
150     55
      ... 
174      1
117      1
192      1
129      1
164      1
Name: count, Length: 67, dtype: int64


In [18]:
print(df_numeric["FastingBS"].value_counts()) ## is a binary variable; statistical IQR does not apply.

FastingBS
0    704
1    214
Name: count, dtype: int64


In [19]:
print(df_numeric["Age"].value_counts()) ## It is already numerical not to apply statistical IQR.

Age
54    51
58    42
55    41
56    38
57    38
52    36
51    35
59    35
62    35
53    33
60    32
48    31
61    31
63    30
50    25
43    24
41    24
46    24
64    22
49    21
65    21
44    19
47    19
45    18
42    18
38    16
39    15
67    15
40    13
66    13
69    13
37    11
35    11
68    10
34     7
74     7
70     7
36     6
32     5
71     5
72     4
29     3
75     3
31     2
33     2
77     2
76     2
28     1
30     1
73     1
Name: count, dtype: int64


# What to do with these cases?

It depends on the project's maturity:

Option When to use Correct (cap at 120) legacy data Input few occurrences Delete critical error record Block ingestion from mature pipelines Create flag always recommended

In [20]:
target_col = "HeartDisease"
cols_numeric_features = [c for c in df_numeric.columns if c != target_col]

df_outliers = df_numeric.drop(columns=["HeartDisease"],  errors="ignore")

# automatically detects binary characters (e.g., 0/1)
binary_cols = [c for c in cols_numeric_features if df_outliers[c].dropna().nunique() <= 2] 

# continuous (where IQR makes sense)
cols_iqr = [c for c in cols_numeric_features if c not in binary_cols and c not in ['Age']] 

In [ ]:
""" OLD check outliers statistic """

for col in df_outliers.columns:
    Q1 = df_raw[col].quantile(0.25)
    Q3 = df_raw[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    outliers = df_raw[(df_raw[col] < lower) | (df_raw[col] > upper)][col]
    print(f"{col}: {len(outliers)} outliers")

In [21]:
print(cols_iqr)

['RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']


In [71]:
""" NEW FUNCTION OUTLIERS """

import numpy as np
import pandas as pd

def iqr_cap_with_flag(df, cols, k=1.5):
    df2 = df_outliers.copy()
    limits = {}

    for col in cols:
        q1 = df2[col].quantile(0.25)
        q3 = df2[col].quantile(0.75)
        iqr = q3 - q1
        low = q1 - k * iqr
        high = q3 + k * iqr

        limits[col] = (low, high)

        df2[f"{col}_outlier_flag"] = ((df2[col] < low) | (df2[col] > high)).astype(int)
        df2[f"{col}_capped"] = df2[col].clip(lower=low, upper=high)

    return df2, limits



df_numeric_treated, iqr_limits = iqr_cap_with_flag(df_outliers, cols=cols_iqr, k=1.5)


In [ ]:
df_numeric.dtypes

In [ ]:
df_outliers.dtypes

In [ ]:
df_numeric_treated.dtypes

# df_numeric_treated

Interpretação de maturidade (nível alto)
O teu dataset agora separa claramente:
| Tipo     | Exemplo     | Tratamento       |
| -------- | ----------- | ---------------- |
| Domínio  | Age         | regra de negócio |
| Binário  | FastingBS   | validação lógica |
| Contínuo | Cholesterol | IQR + cap + flag |

Isso é Data Quality + Feature Engineering, não só ML.


In [75]:
# concatena todas as variáveis
df_encoded = pd.concat([df_numeric_treated, encoded_df], axis=1)

In [ ]:
#check
df_encoded.dtypes

In [ ]:
df_encoded.shape


In [ ]:
df_encoded.isna().sum().sum()


In [ ]:
# adiciona/volta a variável target
df_encoded["HeartDisease"] = df_raw["HeartDisease"]
df_encoded.dtypes

In [ ]:
df_encoded["HeartDisease"].value_counts()


# Verificação de outliers com IQR

IQR = Q3 – Q1
Onde:
Q1 = 25º percentil
Q3 = 75º percentil

Um valor é considerado outlier se estiver fora de: [ Q1 - 1.5*IQR ,  Q3 + 1.5*IQR ]

Regra importante
Outliers só fazem sentido para:

variáveis numéricas contínuas originais
❌ Nunca para colunas one-hot (0/1)
❌ Nunca para o target

Resumo mental (importante)
✔️ df_encoded é o DataFrame certo
✔️ Outliers só nas contínuas reais
❌ Não aplicar IQR em one-hot
❌ Não aplicar IQR no target
✔️ Guardar iqr_limits para produção


# Técnica -> RestingBP
Vou usar o padrão mais recomendado: Winsorization (cap) + flag apenas para colunas numéricas contínuas.

In [ ]:
print(df_encoded["RestingBP"].value_counts()) ### aplicado na função: iqr_cap_with_flag

In [ ]:
print(df_encoded["Cholesterol"].value_counts())

In [ ]:
print(df_encoded["MaxHR"].value_counts())

In [ ]:
print(df_encoded["Oldpeak"].value_counts())

# FIM da análise de Outliers

# ~~Remove Outliers~~ tratamos os outliers em colunas originas númericas e não binárias
Remover outliers reduz linhas do dataset, se fizer isso para muitas colunas e tiver IQR estreito, pode perder muitos dados. Alternativa: substituir outliers por mediana ou limites, por exemplo.

In [ ]:
print(len(df_clean)) ## old ficaram 587 rows

In [ ]:
print(len(df_encoded)) # manteve 918 linhas mesmo tamanho de df_raw OK 

In [ ]:
## nao executado depois o tratamento do outliers, apenas check do antes e depois 
# check total record: ANTES DE TRATAR OS OUTLIERS
print("Total: ",df.shape[0],"Total sem outliers: ", df_clean.shape[0])
reduzidos = df.shape[0] - df_clean.shape[0]
print(f'Foram reduzidos o total de {reduzidos}')

# MODELO

# Separar os dados em treino/teste com stratificação. 

In [93]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

y = df_encoded["HeartDisease"]
X = df_encoded

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


In [ ]:
print(X_train.dtypes)


In [95]:
# Selecionar somente colunas numéricas
numeric_cols = X_train.select_dtypes(include=['number']).columns

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[numeric_cols])
X_test_scaled = scaler.transform(X_test[numeric_cols])

# Aplicar um modelo K-Nearest Neighbors (KNN).

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)


In [ ]:
y_train.value_counts()

In [ ]:
print(y_train.value_counts())
print(y_test.value_counts())


# Avaliar o modelo com:
 - Matriz de confusão
 - o Acurácia, Precisão, Revocação e F1-score

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# predict: previsões com os dados de teste
y_pred = knn.predict(X_test_scaled)
cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap="Blues")    
plt.title("Matriz de Confusão")
plt.show()

# explicação:
TN (True Negative): O modelo previu 0 e o rótulo real era 0 → previsão correta de negativo.

FN (False Negative): O modelo previu 0 mas o rótulo real era 1 → "falha na deteção".

FP (False Positive): O modelo previu 1 mas o rótulo real era 0 → "falso alarme".

TP (True Positive): O modelo previu 1 e o rótulo real era 1 → previsão correta de positivo.


| Real \ Previsto | 0 (Previsto) | 1 (Previsto) |
|-----------------|--------------|--------------|
| 0 (Real)        | TN           | FP           |
| 1 (Real)        | FN           | TP           |


In [ ]:
print("Matriz de Confusão:\n", cm)

In [ ]:
print("\nRelatório de Classificação:\n" , classification_report(y_test, y_pred))

# EXPLICAÇÃO 1
### Após oneHotEnconder
### Apenas com as variáveis numericas

# Análise de KNN (baseado no pdf 06-FT03)
``` knn = KNeighborsClassifier(n_neighbors=5) ```

Na primeira aplicação com hiper parametros padrão e número de clusters igual 5, obtivemos:

**Total da amostra: 918 rows (100%)**

- TN (True Negative): 76% 
- FP (False Positive): 6% 
- FN (False Negative): 3% 
- TP (True Positive): 99% 


Ou seja mais de 175% do total de TN + TP (76% TN + 99% TP) do nosso modelo treinou corretamente.

Em contra partida, tivemos 3(FN) linhas da amostra classificado incorretamente, no caso do problema em questão pode ser fatal, vamos considerar como falha na deteção da doença(FN), pelo modelo.

Como ponto de melhoria: diminuir os 'False Negative'.

* Com relação a outras métricas:




1. Accuracy: obtivemos *0.936*, podemos considerar ótimo para o modelo conseguiu predizer 94% dos casos na amostra, porém é importante ajustar para conseguir adicionar os 3% (FN) restantes como TP.

Melhores parâmetros: {'knn__metric': 'manhattan', **'knn__n_neighbors': 11**, 'knn__weights': 'distance'}
Melhor accuracy (validação): 0.936 
Accuracy no teste: 0.951



| Classe | Precision | Recall | F1-score | Support |
|--------|-----------|--------|----------|---------|
| 0      | 0.96      | 0.93   | 0.94     | 82      |
| 1      | 0.94      | 0.97   | 0.96     | 102     |
| **Accuracy** |           |        | **0.95** | 184     |
| **Macro avg** | 0.95      | 0.95   | 0.95     | 184     |
| **Weighted avg** | 0.95      | 0.95   | 0.95     | 184     |

Pessoas saudáveis:
2. Na Precisão de positivos e está certo tivemos 96%, dentre os quais 4% estão classificados com imprecisão, não tem 100% de 'certeza', como 0 (podem ou não estarem saudáveis), neste caso foram marcados como saudáveis mas podem ser que estejam <u>doentes</u>.

3. recall
Recall = Verdadeiros Positivos / (Verdadeiros Positivos + Falsos Negativos)
O que significa que todos os exemplos que realmente eram classe 0, pessoas saudáveis, o modelo conseguiu identificar 93%. deixando apenas 7% como falsos saudáveis.

4. F-score
F1 = 2 * (precision * recall) / (precision + recall)
o que significa que essa métrica mede o equilíbrio. Evita que um modelo com recall alto e precision baixa (ou vice-versa) pareça bom.
em resumo, precisamos ter as duas métricas balanceadas para de facto ter um modelo com boa predição.
f.Score = 94% e recall = 93% (temos equilíbrio)

Médias:
Macro Avg = 0.95 pode representar um modelo optimo.
Weighted Avg = 0.95, semelhante a accuracy, logo, podemos dizer que as classes estão balanceadas entre elas.
➡️ Se weighted = accuracy, normalmente as classes não estão muito desbalanceadas.
Weighted Avg  =  95% e comparanco com a accuracy = 94% (temos balanceamanto)

conclusão
Encrontramos um ponto crítico de possível melhoria para o futuro:

1. Ponto crítico de melhoria, para os recall em classe de pessoas doentes, positivas para ataque. tivemos recall = 94%, ou seja deixou de predizer corretamente 6% dos doentes, o que pode ser fatal. 

1.1 Sugestão de solução: melhorar o balanceamento de pessoas doentes (foi feito, aplicamos o OneHotEncoder), ou seja, aumentar a quantidade de doentes na amostra de treinamento do modelo. Já para deixar os padrões de pessoas doentes, precisamos confirmar se temos mais atributos que podem explicam melhor a classe 1(pessoas doentes).   DONE 
1.2 Tratamos os outlires com as técnicas de  flags, tratamos apenas variáveis numéricas originas no df_raw e não binárias. DONE 

resultado: após melhorias 1.1 e 1.2 os TP subiram para 99% de detecção. sugestão para nova aplicação do KNN será com 11 K.

# ~~EXPLICAÇÃO 2~~ antes do onhotencode
### Antes oneHotEnconder
### Com as variáveis numericas + categorical transformadas em numericas com OneHotEncoder
# Análise de KNN (baseado no pdf 06-FT03)
``` knn = KNeighborsClassifier(n_neighbors=5) ```

Na primeira aplicação com hiper parametros padrão e número de clusters igual 5, obtivemos:
TP (True Positive): 59 do total de 587 amostras
TN (True Negative): 16 do total de 587 amostras
FP (False Positive): 19 do total de 587 amostras
FN (False Negative): 31 do total de 587 amostras

ou seja mais de 50% do nosso modelo treinou corretamente. Em contra partida, tivemos 31 linhas da amostra classificado incorretamente, no caso do problema em questão pode ser fatal, vamos considerar como falha na deteção da doença pelo modelo.

Com relação a outras métricas:
1. Accuracy: obtivemos *0.755*, podemos considerar  bom para o modelo conseguiu predizer 70 % dos casos na amostra, porém é importate ajustar para conseguir adicionar os 30% restantes como TP.
Melhor accuracy (validação): 0.755 
Accuracy no teste: 0.703

Pessoas saudáveis:

| Classe        | Precision | Recall | F1-Score | Support |
|---------------|-----------|--------|----------|---------|
| **0**         | 0.73      | 0.76   | 0.75     | 68      |
| **1**         | 0.66      | 0.62   | 0.64     | 50      |
| **Accuracy**  | —         | —      | 0.70     | 118     |
| **Macro Avg** | 0.70      | 0.69   | 0.69     | 118     |
| **Weighted Avg** | 0.70  | 0.70   | 0.70     | 118     |


2. Na Precisão de posotivos e está certo tivemos 73 %

3. recall
Recall = Verdadeiros Positivos / (Verdadeiros Positivos + Falsos Negativos)
O que significa que todos os exemplos que realmente eram classe 0, pessoas saudáveis, o modelo conseguiu identificar 76%. deixando apenas 24% como falsos doentes.

4. F-score
F1 = 2 * (precision * recall) / (precision + recall)
o que significa que essa métrica mede o equilíbrio. Evita que um modelo com recall alto e precision baixa (ou vice-versa) pareça bom.
em resumo, precisamos ter as duas métricas balanceadas para de facto ter um modelo com boa predição.

Médias:
Macro Avg = 0.69 pode representar um modelo moderado.
Weighted Avg = 0.70, semelhante a accuracy, logo, podemos dizer que as classes estão balanceadas entre elas.
➡️ Se weighted = accuracy, normalmente as classes não estão muito desbalanceadas.

conclusão
Encrontramos um ponto crítico de possível melhoria para futura:

1. Ponto crítico de melhoria, para os recall em classe de pessoas doentes, positivas para ataque. tivemos recall = 62& ou seja deixou de predizer corretamente 38% dos doentes, o que pode ser fatal. 
1.1 Sugestão de solução: melhorar o balanceamento de pessoas doentes, ou seja, aumentar a quantidade de doentes na amostra de treinamento do modelo. Já para deixar os padrões de pessoas doentes, precisamos confirmar se temos mais atributos que podem explicam melhor a classe 1(pessoas doentes).   

# Otimizar o número de vizinhos com GridSearchCV

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

#pipeline
pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier())
])

# Hiper parametros
param_grid = {
    "knn__n_neighbors": [3, 5, 7, 9, 11],
    "knn__weights": ["uniform", "distance"],
    "knn__metric": ["euclidean", "manhattan"]
}

# otimização de vizinhos
grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=5
)

grid.fit(X_train_scaled, y_train)

print("Melhores parâmetros:", grid.best_params_)
print(f"Melhor accuracy (validação): {grid.best_score_:.3f} ")

# best_model = grid.best_estimator_
# y_pred = best_model.predict(X_test_scaled)

# accuracy_score: percentagem de classificações corretas
test_acc = accuracy_score(y_test, y_pred)
print(f"Accuracy no teste: {test_acc:.3f}")

# Comparar o desempenho antes e depois da otimização.

# Análise de KNN (baseado no pdf 06-FT03)
## knowledge
``` knn = KNeighborsClassifier(n_neighbors=5) ```

Na primeira aplicação com hiper parametros padrão e número de clusters igual 5, obtivemos:
TP (True Positive): 59 do total de 587 amostras
TN (True Negative): 16 do total de 587 amostras
FP (False Positive): 19 do total de 587 amostras
FN (False Negative): 31 do total de 587 amostras

ou seja mais de 50% do nosso modelo treinou corretamente. Em contra partida, tivemos 31 linhas da amostra classificado incorretamente, no caso do problema em questão pode ser fatal, vamos considerar como falha na deteção da doença pelo modelo.

Com relação a outras métricas:
1. Accuracy: obtivemos *0.755*, podemos considerar  bom para o modelo conseguiu predizer 70 % dos casos na amostra, porém é importate ajustar para conseguir adicionar os 30% restantes como TP.

Pessoas saudáveis:
2. Na Precisão de posotivos e está certo tivemos 73 %

3. recall
Recall = Verdadeiros Positivos / (Verdadeiros Positivos + Falsos Negativos)
O que significa que todos os exemplos que realmente eram classe 0, pessoas saudáveis, o modelo conseguiu identificar 76%. deixando apenas 24% como falsos doentes.

4. F-score
F1 = 2 * (precision * recall) / (precision + recall)
o que significa que essa métrica mede o equilíbrio. Evita que um modelo com recall alto e precision baixa (ou vice-versa) pareça bom.
em resumo, precisamos ter as duas métricas balanceadas para de facto ter um modelo com boa predição.

Médias:
Macro Avg = 0.69 pode representar um modelo moderado.
Weighted Avg = 0.70, semelhante a accuracy, logo, podemos dizer que as classes estão balanceadas entre elas.
➡️ Se weighted = accuracy, normalmente as classes não estão muito desbalanceadas.

conclusão
Encrontramos um ponto crítico de possível melhoria para futura:

1. Ponto crítico de melhoria, para os recall em classe de pessoas doentes, positivas para ataque. tivemos recall = 62& ou seja deixou de predizer corretamente 38% dos doentes, o que pode ser fatal. 
1.1 Sugestão de solução: melhorar o balanceamento de pessoas doentes, ou seja, aumentar a quantidade de doentes na amostra de treinamento do modelo. Já para deixar os padrões de pessoas doentes, precisamos confirmar se temos mais atributos que podem explicam melhor a classe 1(pessoas doentes).

# CONTEXTO PARA explicação:
TP (True Positive): O modelo previu 1 e o rótulo real era 1 → previsão correta de positivo.

TN (True Negative): O modelo previu 0 e o rótulo real era 0 → previsão correta de negativo.

FP (False Positive): O modelo previu 1 mas o rótulo real era 0 → "falso alarme".

FN (False Negative): O modelo previu 0 mas o rótulo real era 1 → "falha na deteção".


| Real \ Previsto | 0 (Previsto) | 1 (Previsto) |
|-----------------|--------------|--------------|
| 0 (Real)        | TN           | FP           |
| 1 (Real)        | FN           | TP           |


# resultado do classification_report
Relatório de Classificação:

classe 0 => Saudável (falso)

classe 1 => doente (positivo)

# Sem oneHotEncoder

| Classe        | Precision | Recall | F1-Score | Support |
|---------------|-----------|--------|----------|---------|
| **0**         | 0.73      | 0.76   | 0.75     | 68      |
| **1**         | 0.66      | 0.62   | 0.64     | 50      |
| **Accuracy**  | —         | —      | 0.70     | 118     |
| **Macro Avg** | 0.70      | 0.69   | 0.69     | 118     |
| **Weighted Avg** | 0.70  | 0.70   | 0.70     | 118     |


# Com oneHotEncoder
| Classe        | Precision | Recall | F1-Score | Support |
|---------------|-----------|--------|----------|---------|
| **0**         | 0.97      | 0.87   | 0.91     | 68      |
| **1**         | 0.84      | 0.96   | 0.90     | 50      |
| **Accuracy**  | —         | —      | 0.91     | 118     |
| **Macro Avg** | 0.90      | 0.91   | 0.91     | 118     |
| **Weighted Avg** | 0.91  | 0.91   | 0.91     | 118     |


# Prever a condição de um novo paciente
*com valores fictícios

# Entendendo os campos:
| Campo           | Descrição                                                    | Exemplo | Tipo        | Por extenso / Significado                                  |
|-----------------|--------------------------------------------------------------|---------|-------------|-------------------------------------------------------------|
| Age             | Idade do paciente                                            | 55      | numérica    | 55 anos                                                     |
| Sex             | Sexo do paciente                                             | M       | categórica  | M = masculino, F = feminino                                 |
| ChestPainType   | Tipo de dor no peito                                         | ATA     | categórica  | ATA = angina atípica; ASY = assintomático; TA = típica; NAP = não anginosa |
| RestingBP       | Pressão arterial em repouso (mm Hg)                           | 130     | numérica    | 130 mm Hg                                                   |
| Cholesterol     | Colesterol sérico total (mg/dl)                               | 245     | numérica    | 245 mg/dl                                                   |
| FastingBS       | Glicemia em jejum > 120 mg/dl?                                | 0       | categórica  | 0 = normal; 1 = alta (acima de 120 mg/dl)                   |
| RestingECG      | Resultado do eletrocardiograma em repouso                     | ST      | categórica  | Normal / Anomalia ST-T / Hipertrofia Ventricular Esquerda  |
| MaxHR           | Frequência cardíaca máxima atingida                           | 150     | numérica    | 150 bpm                                                     |
| ExerciseAngina  | Angina induzida por exercício                                 | N       | categórica  | N = não apresentou; Y = apresentou angina                   |
| Oldpeak         | Depressão do segmento ST por exercício                        | 1.0     | numérica    | Depressão ST de 1.0 mm                                      |
| ST_Slope        | Inclinação do segmento ST no pico do exercício                | Down    | categórica  | Up = subida; Flat = plano; Down = descida                   |
| HeartDisease    | Presença de doença cardíaca (TARGET)                          | 1       | categórica  | 1 = possui doença cardíaca; 0 = não possui                  |



# Reaplicar o modelo filtrando as colunas originais e usando as colunas tratadas dos outliers

# knowledge
* Entre pessoas experientes em Data Stewardship (com foco em governança + explicabilidade + robustez), a opção mais recomendada costuma ser:

✅ Usar _capped + _outlier_flag e NÃO usar as colunas originais (não capped) no modelo.
(equivalente à tua Opção A, mas “sem duplicar” a mesma variável duas vezes.)

Por quê essa é a mais defendida?

- Robustez: o modelo não “explode” por valores extremos.
- Explicabilidade: a flag diz claramente “esse caso era extremo”.
- Governança: fica transparente o que foi tratado e quando (útil para auditoria e stakeholders).
- Menos ruído: evitar manter original + capped juntos reduz colinearidade e confusão na interpretação.

Quando eu NÃO recomendaria essa opção?
- Se o outlier é um evento raro porém super informativo (ex.: fraude, picos reais) e você quer que o modelo “sinta” a magnitude total. Aí você pode manter o original também — mas isso é uma decisão consciente, não padrão.

In [105]:
# Recomendado: usar _capped + _outlier_flag e remover as colunas originais contínuas
target = "HeartDisease"
orig_continuous = ["RestingBP", "Cholesterol", "MaxHR", "Oldpeak"]

capped_cols = [c for c in df_encoded.columns if c.endswith("_capped")]
flag_cols  = [c for c in df_encoded.columns if c.endswith("_outlier_flag")]

# base = tudo que não é target, não é original contínua, não é capped/flag
base_cols = [
    c for c in df_encoded.columns
    if c != target
    and c not in orig_continuous
    and not c.endswith("_capped")
    and not c.endswith("_outlier_flag")
]

X = df_encoded[base_cols + capped_cols + flag_cols]
y = df_encoded[target]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(X_train_scaled, y_train)

In [ ]:
# classificação 
y_train.value_counts()

In [ ]:
print(y_train.value_counts())
print(y_test.value_counts())

In [ ]:
# predict: previsões com os dados de teste
y_pred = knn.predict(X_test_scaled)
cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap="Blues")    
plt.title("Matriz de Confusão")
plt.show()

| Real \ Previsto | 0 (Previsto) | 1 (Previsto) |
|-----------------|--------------|--------------|
| 0 (Real)        | TN           | FP           |
| 1 (Real)        | FN           | TP           |


In [ ]:
print("Matriz de Confusão:\n", cm)

In [ ]:
print("\nRelatório de Classificação:\n" , classification_report(y_test, y_pred))

# OTIMIZAÇÃO GRID SEARCH CV

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

#pipeline
pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier())
])

# Hiper parametros
param_grid = {
    "knn__n_neighbors": [3, 5, 7, 9, 11],
    "knn__weights": ["uniform", "distance"],
    "knn__metric": ["euclidean", "manhattan"]
}

# otimização de vizinhos
grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=5
)

grid.fit(X_train_scaled, y_train)

print("Melhores parâmetros:", grid.best_params_)
print(f"Melhor accuracy (validação): {grid.best_score_:.3f} ")

# best_model = grid.best_estimator_
# y_pred = best_model.predict(X_test_scaled)

# accuracy_score: percentagem de classificações corretas
test_acc = accuracy_score(y_test, y_pred)
print(f"Accuracy no teste: {test_acc:.3f}")

# NOVA PREDIÇÃO

In [ ]:
X_train.columns

In [ ]:
X_train.head(1)

In [ ]:
df_encoded.columns

In [ ]:
import pandas as pd


# 0) Novo paciente (formato original)
novo_paciente = pd.DataFrame([{
    "Age": 58,
    "Sex": "M",
    "ChestPainType": "ATA",
    "RestingBP": 138,
    "Cholesterol": 240,
    "FastingBS": 0,
    "RestingECG": "ST",
    "MaxHR": 160,
    "ExerciseAngina": "N",
    "Oldpeak": 1.4,
    "ST_Slope": "Flat"
}])


# 1) One-hot encoding (usar o MESMO encoder já fitado)
df_categorical = novo_paciente.select_dtypes(exclude=["number"])
encoded_arr = encoder.transform(df_categorical)

encoded_df_new = pd.DataFrame(
    encoded_arr,
    columns=encoder.get_feature_names_out(df_categorical.columns),
    index=novo_paciente.index
)


# 2) Numéricas + capped/flags (usar os MESMOS limites iqr_limits do treino)
df_numeric = novo_paciente.select_dtypes(include=["number"]).copy()

for col in ["RestingBP", "Cholesterol", "MaxHR", "Oldpeak"]:
    low, high = iqr_limits[col]
    df_numeric[f"{col}_outlier_flag"] = ((df_numeric[col] < low) | (df_numeric[col] > high)).astype(int)
    df_numeric[f"{col}_capped"] = df_numeric[col].clip(lower=low, upper=high)

df_numeric_final = df_numeric[[
    "Age", "FastingBS",
    "RestingBP_capped", "Cholesterol_capped", "MaxHR_capped", "Oldpeak_capped",
    "RestingBP_outlier_flag", "Cholesterol_outlier_flag", "MaxHR_outlier_flag", "Oldpeak_outlier_flag"
]]


# 3) Montar X do novo paciente e alinhar com as colunas do treino
novo_X = pd.concat([df_numeric_final, encoded_df_new], axis=1)

# alinhar com as colunas que o modelo espera (X_train.columns)
novo_X = novo_X.reindex(columns=X_train.columns, fill_value=0)


# 4) Escalar com o MESMO scaler (alinhar com as colunas do scaler)
if hasattr(scaler, "feature_names_in_"):
    cols_scaler = list(scaler.feature_names_in_)
else:
    cols_scaler = list(X_train.columns)  # fallback: use a ordem do treino

novo_X_for_scaler = novo_X.reindex(columns=cols_scaler, fill_value=0)
novo_X_scaled = scaler.transform(novo_X_for_scaler)


# 5) Predição com o modelo treinado (GridSearchCV)
pred_novo = grid.predict(novo_X_scaled)

print(pred_novo[0])
print(f"Risco previsto: {'Doente' if pred_novo[0] == 1 else 'Saudável'}")
